In [1]:
import numpy as np
import pandas as pd

In [2]:
cols = ['primary_label', 'secondary_labels', 'record_name', 'source']


In [3]:
train = pd.read_csv('../input/birdclef-2024/train_metadata.csv')
train

train['record'] = [filename.split('/')[1] for filename in train.filename]
train['record_name'] = [record.split('.')[0] for record in train.record]

train['secondary_labels'] = [eval(secondary_labels) for secondary_labels in train['secondary_labels']]
train['source'] = 'bc24'
train[cols]

,primary_label,secondary_labels,record_name,source
0,asbfly,[],XC134896,bc24
1,asbfly,[],XC164848,bc24
2,asbfly,[],XC175797,bc24
3,asbfly,[],XC207738,bc24
4,asbfly,[],XC209218,bc24
...,...,...,...,...
24454,zitcis1,[],XC845747,bc24
24455,zitcis1,[],XC845817,bc24
24456,zitcis1,[],XC856176,bc24
24457,zitcis1,[],XC856723,bc24


In [8]:
train_additional = pd.read_csv('../input/birdclef-2024/train_additional.csv')
train_additional
train_additional['source'] = 'bc00'
train_additional['record_name'] = [record.split('.')[0] for record in train_additional.record]
train_additional['primary_label'] = train_additional['species']
train_additional['secondary_labels'] = [[] for _ in train_additional['primary_label']]
train_additional = train_additional[~train_additional.record_name.isin(train.record_name.unique())].reset_index(drop=True)
train_additional[cols]

,primary_label,secondary_labels,record_name,source
0,junowl1,[],XC369236,bc00
1,junowl1,[],XC161242,bc00
2,junowl1,[],XC187332,bc00
3,junowl1,[],XC319956,bc00
4,junowl1,[],XC369233,bc00
...,...,...,...,...
5867,commyn,[],XC472545,bc00
5868,commyn,[],XC79539,bc00
5869,commyn,[],XC871100,bc00
5870,commyn,[],XC102124,bc00


In [12]:
train_20 = pd.read_csv('../input/Birdclef20/train.csv')
train_20

train_20['record_name'] = [record.split('.')[0] for record in train_20.filename]
train_20['primary_label'] = train_20['ebird_code']
train_20['source'] = 'bc20'
train_20 = train_20[~train_20.record_name.isin(train.record_name.unique())].reset_index(drop=True)
train_20 = train_20[train_20.primary_label.isin(train.primary_label.unique())].reset_index(drop=True)
train_20[cols]

,primary_label,secondary_labels,record_name,source
0,barswa,[],XC139171,bc20
1,barswa,[],XC139728,bc20
2,barswa,[],XC142632,bc20
3,barswa,[],XC142731,bc20
4,barswa,[],XC156257,bc20
...,...,...,...,...
167,houspa,[],XC495694,bc20
168,houspa,[],XC501121,bc20
169,houspa,[],XC503268,bc20
170,houspa,[],XC503474,bc20


In [13]:
df = train_20[['species', 'ebird_code']].drop_duplicates().sort_values('species').reset_index(drop=True)
df

species2code = {species : code for species, code in zip(df.species, df.ebird_code)}
species2code

def process_secondary_labels(secondary_labels, species2code):
    secondary_labels = eval(secondary_labels)
    labels = []
    for secondary in secondary_labels:
        label = species2code.get(secondary.split('_')[1], '')
        if label != '':
            labels.append(label)
    return labels

train_20['secondary_labels'] = [process_secondary_labels(secondary_labels, species2code) for secondary_labels in train_20['secondary_labels']]

In [14]:
train_21 = pd.read_csv('../input/Birdclef21/train_metadata.csv')
train_21
train_21['secondary_labels'] = [eval(secondary_labels) for secondary_labels in train_21['secondary_labels']]
train_21['record_name'] = [record.split('.')[0] for record in train_21.filename]
train_21['source'] = 'bc21'
train_21 = train_21[~train_21.record_name.isin(train.record_name.unique())].reset_index(drop=True)
train_21 = train_21[train_21.primary_label.isin(train.primary_label.unique())].reset_index(drop=True)
train_21[cols]

,primary_label,secondary_labels,record_name,source
0,barswa,[],XC11464,bc21
1,barswa,[],XC125026,bc21
2,barswa,[],XC126103,bc21
3,barswa,"[marwre, sonspa]",XC129647,bc21
4,barswa,[],XC132406,bc21
...,...,...,...,...
842,houspa,[],XC614109,bc21
843,houspa,[],XC616097,bc21
844,houspa,"[gamqua, whcspa]",XC616577,bc21
845,houspa,"[gamqua, rewbla]",XC616579,bc21


In [15]:
train_22 = pd.read_csv('../input/Birdclef22/train_metadata.csv')
train_22['secondary_labels'] = [eval(secondary_labels) for secondary_labels in train_22['secondary_labels']]
train_22['record_name'] = [record.split('/')[1].split('.')[0] for record in train_22.filename]
train_22['source'] = 'bc22'
train_22 = train_22[~train_22.record_name.isin(train.record_name.unique())].reset_index(drop=True)
train_22 = train_22[train_22.primary_label.isin(train.primary_label.unique())].reset_index(drop=True)
train_22[cols]

,primary_label,secondary_labels,record_name,source
0,bcnher,[],XC119210,bc22
1,bcnher,[],XC123806,bc22
2,bcnher,[],XC134969,bc22
3,bcnher,[],XC153095,bc22
4,bcnher,[],XC170797,bc22
...,...,...,...,...
794,houspa,[],XC660840,bc22
795,houspa,[],XC662681,bc22
796,houspa,[],XC665867,bc22
797,houspa,[],XC667230,bc22


In [17]:
train_23 = pd.read_csv('../input/Birdclef23/train_metadata.csv')
train_23['secondary_labels'] = [eval(secondary_labels) for secondary_labels in train_23['secondary_labels']]
train_23['record_name'] = [record.split('/')[1].split('.')[0] for record in train_23.filename]
train_23['source'] = 'bc23'
train_23 = train_23[~train_23.record_name.isin(train.record_name.unique())].reset_index(drop=True)
train_23 = train_23[train_23.primary_label.isin(train.primary_label.unique())].reset_index(drop=True)
train_23[cols]

,primary_label,secondary_labels,record_name,source
0,barswa,[],XC113914,bc23
1,barswa,[],XC129647,bc23
2,barswa,[],XC132406,bc23
3,barswa,[],XC133096,bc23
4,barswa,[],XC135474,bc23
...,...,...,...,...
1127,woosan,[],XC739000,bc23
1128,woosan,[],XC745719,bc23
1129,woosan,[],XC746579,bc23
1130,woosan,[],XC747214,bc23


In [18]:
all_train = pd.concat([train_20[cols], 
                       train_21[cols], 
                       train_22[cols], 
                       train_23[cols],
                       train_additional[cols],
                       train[cols],
                       ]).reset_index(drop=True)
all_train

,primary_label,secondary_labels,record_name,source
0,barswa,[],XC139171,bc20
1,barswa,[],XC139728,bc20
2,barswa,[],XC142632,bc20
3,barswa,[],XC142731,bc20
4,barswa,[],XC156257,bc20
...,...,...,...,...
33276,zitcis1,[],XC845747,bc24
33277,zitcis1,[],XC845817,bc24
33278,zitcis1,[],XC856176,bc24
33279,zitcis1,[],XC856723,bc24


In [19]:
all_train = all_train.sort_values(['primary_label', 'source'], ascending=False).reset_index(drop=True)
all_train = all_train.drop_duplicates('record_name')
all_train['rank'] = all_train.groupby('primary_label').source.rank(method='first', ascending=False)
all_train

,primary_label,secondary_labels,record_name,source,rank
0,zitcis1,[zitcis1],XC124995,bc24,1.0
1,zitcis1,[],XC125068,bc24,2.0
2,zitcis1,[bkwsti],XC125875,bc24,3.0
3,zitcis1,[],XC126356,bc24,4.0
4,zitcis1,[],XC126357,bc24,5.0
...,...,...,...,...,...
33276,asbfly,[],XC709719,bc00,151.0
33277,asbfly,[],XC832874,bc00,152.0
33278,asbfly,[],XC65352,bc00,153.0
33279,asbfly,[],XC779042,bc00,154.0


In [21]:
all_train.source.value_counts()

bc24    24440
bc00     5852
bc23     1132
bc22      650
bc21      477
bc20       17
Name: source, dtype: int64

In [22]:
all_train = all_train[all_train.source != 'bc24'].reset_index(drop=True)
all_train

all_train = all_train[all_train['rank'] <= 500].reset_index(drop=True)
all_train



,primary_label,secondary_labels,record_name,source,rank
0,yebbul3,[],XC369020,bc00,31.0
1,yebbul3,[],XC311119,bc00,32.0
2,yebbul3,[],XC68513,bc00,33.0
3,yebbul3,[],XC369021,bc00,34.0
4,yebbul3,[],XC311117,bc00,35.0
...,...,...,...,...,...
5936,asbfly,[],XC709719,bc00,151.0
5937,asbfly,[],XC832874,bc00,152.0
5938,asbfly,[],XC65352,bc00,153.0
5939,asbfly,[],XC779042,bc00,154.0


In [23]:
all_train.to_csv('../input/birdclef-2024/all_train.csv', index=False)

In [24]:
from pathlib import Path
from tqdm import tqdm
import librosa
sr = 32000

def load_audio(record_name, primary_label, source):
    if source == 'bc20':
        pathname = Path('../../Birdclef20/input/train_audio/') / primary_label / (record_name + '.mp3')
    elif source == 'bc21':
        pathname = Path('../../Birdclef21/input/train_short_audio/') / primary_label / (record_name + '.ogg')
    elif source == 'bc22':
        pathname = Path('../../Birdclef22/train_audio/') / primary_label / (record_name + '.ogg')
    elif source == 'bc23':
        pathname = Path('../../Birdclef23/train_audio/') / primary_label / (record_name + '.ogg')
    elif source == 'bc00':
        pathname = Path('../input/birdclef2024-additional-mp3/additional_audio/') / primary_label / (record_name + '.mp3')
    audio = librosa.load(pathname, sr=32000)[0].astype(np.float32)
    return audio

In [25]:
lengths = []
for record_name, primary_label, source in zip(tqdm(all_train.record_name), all_train.primary_label, all_train.source):
    if source in ['bc00', 'bc20', 'bc21', 'bc22', 'bc23']:
        audio = load_audio(record_name, primary_label, source)
        lengths.append(len(audio))
        save_path = Path('../input') / 'birdclef_data' / primary_label
        np.save(save_path / ('first10_' + record_name), audio[: 10 * sr])
        np.save(save_path / ('last10_' + record_name), audio[-10 * sr : ])

100%|██████████| 5941/5941 [03:07<00:00, 31.68it/s]


In [26]:
len(lengths)

5941